In [46]:
import cv2
import numpy as np
import time

In [47]:
#np.linalg.norm([-2,2])

In [69]:
#Importar video

fileName='PC124surfTrim203-2.mp4'
cap = cv2.VideoCapture(fileName)


cantidadFrames=5
#por que se usa un tipo de video sin ningun tipo de estabilizacion, 
#el codigo funciona mejor con una cantidad muy limitada de frames


skipframes=300
#saltea una cantidad x de frames al inicio

# Parametros para lk_track

# problemas actuales:
*detectar cual marcador corresponde con el siguiente en la deteccion

*analizar hacia adonde se dirige el flujo

In [70]:
mask=0
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))

#numero de frames que saltea al inicio
while(skipframes):
    cap.read()
    skipframes-=1
    
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

puntos=np.zeros([p0.shape[0],cantidadFrames+2,2])#arreglo de puntos para analizar

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
actualFrame=0
while(actualFrame<=cantidadFrames):
    actualFrame+=1
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
    #anaizar correspodencia entre puntos buenos viejos y nuevos
    #limito a un maximo de "100(x)" los puntos
    if actualFrame==1:
        for a in range (0,good_old.shape[0]):#optimizar
            puntos[a,0,:]=good_old[a]
                
    cantidadPuntosNuevos,v =good_new.shape
    for pos in range (0, cantidadPuntosNuevos):
        for pospuntos in range (0,puntos.shape[0]):
            x=puntos[pospuntos,actualFrame-1,0]-good_new[pos,0]#distancia en x
            y=puntos[pospuntos,actualFrame-1,1]-good_new[pos,1]#distancia en y
            dist=np.linalg.norm([x,y])#distancia entre los 2 puntos
            if dist<10:#si la distancia es menor a 10 "pixeles" coloca el punto como coincidente con el anterior
                puntos[pospuntos,actualFrame]=good_new[pos]
                
                #if puntos[pospuntos,actualFrame]==[0,0]:
                #    puntos[pospuntos,actualFrame]=good_new[pos]
                #else:
                #    comparar cual tiene distancia minima con el punto anterior
                    
    
    
    
    
    #np.append(puntos,p1,actualFrame) #arreglo de puntos destacados 
    #time.sleep(2)#pausa para ver el programa
    
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()

# Analisis de puntos obtenidos

In [88]:
puntos.shape

(100, 7, 2)

In [110]:
#limpia ordena los puntos y quita las filas con solo 0
puntosLimp=puntos[:,~np.all(puntos == 0, axis=0)]
puntosLimp2=puntosLimp[~np.all(puntosLimp == 0, axis=1)]
puntosLimp2.shape#los ordeno en un arreglo de 2 dimension para poder guardar el archivo mas facil

(98, 14)

In [114]:
#calculo del angulo y modulo de desplazamiento de los puntos
angulo=np.zeros(puntosLimp2.shape[0])
modulo=np.zeros(puntosLimp2.shape[0])

for pos in range (2, puntosLimp2.shape[0]):
    contador=0
    mod=0
    ang=0
    for elemento in range (2,puntosLimp2.shape[1],2):
        if puntosLimp2[pos,elemento]!=0:
            contador+=1
            x=puntosLimp2[pos,elemento-2]-puntosLimp2[pos,elemento]
            y=puntosLimp2[pos,elemento-1]-puntosLimp2[pos,elemento+1]
            mod+=np.linalg.norm([x,y])#distancia entre los 2 puntos
            ang+=np.angle(x+y*1j,deg=True)
           
    if contador!=0:
        modulo[pos]=mod/contador
        angulo[pos]=ang/contador



In [118]:
anguloprom=np.average(angulo)
anguloprom

22.20105861946757

In [119]:
moduloprim=np.average(modulo)
moduloprim

6.780783225249452

# Calcular errores medios cuadraticos

In [104]:
puntosLimp=puntos[~np.all(puntos == 0, axis=2)]
puntosLimp.shape

(339, 2)

In [106]:
np.savetxt('puntos.out', puntos[1,:,:], delimiter=',')
np.savetxt('puntosLimp.out', puntosLimp[:,:], delimiter=',')

In [84]:
a=np.array([[2,2,2],[0,0,0],[1,1,1]])
a[~np.all(a == 0, axis=1)]

array([[2, 2, 2],
       [1, 1, 1]])